In [1]:
import pandas as pd;import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sidetable as stb

from IPython.display import HTML, display
import tabulate
%matplotlib inline

## Read the data to Pandas Data Frame

In [4]:
df = pd.read_excel("CustomerChurnData.xlsx")
df.head()

,AccountID,Churn,Tenure,City_Tier,CC_Contacted_LY,Payment,Gender,Service_Score,Account_user_count,account_segment,CC_Agent_Score,Marital_Status,rev_per_month,Complain_ly,rev_growth_yoy,coupon_used_for_payment,Day_Since_CC_connect,cashback,Login_device
0,20000,1,4,3.0,6.0,Debit Card,Female,3.0,3,Super,2.0,Single,9,1.0,11,1,5,159.93,Mobile
1,20001,1,0,1.0,8.0,UPI,Male,3.0,4,Regular Plus,3.0,Single,7,1.0,15,0,0,120.9,Mobile
2,20002,1,0,1.0,30.0,Debit Card,Male,2.0,4,Regular Plus,3.0,Single,6,1.0,14,0,3,NaN,Mobile
3,20003,1,0,3.0,15.0,Debit Card,Male,2.0,4,Super,5.0,Single,8,0.0,23,0,3,134.07,Mobile
4,20004,1,0,1.0,12.0,Credit Card,Male,2.0,3,Regular Plus,5.0,Single,3,0.0,11,1,3,129.6,Mobile


## Data Report

In [5]:
#Shape of the Data
print('****Shape of the hotel data****')
print('********************************')
print('No of rows\t:\t{}\nNo of columns\t:\t{}'.format(df.shape[0],df.shape[1]))

****Shape of the hotel data****
********************************
No of rows	:	11260
No of columns	:	19


In [9]:
print('****Show infomation of the train data****')
print('***********************************')
df.info()

****Show infomation of the train data****
***********************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11260 entries, 0 to 11259
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   AccountID                11260 non-null  int64  
 1   Churn                    11260 non-null  int64  
 2   Tenure                   11158 non-null  object 
 3   City_Tier                11148 non-null  float64
 4   CC_Contacted_LY          11158 non-null  float64
 5   Payment                  11151 non-null  object 
 6   Gender                   11152 non-null  object 
 7   Service_Score            11162 non-null  float64
 8   Account_user_count       11148 non-null  object 
 9   account_segment          11163 non-null  object 
 10  CC_Agent_Score           11144 non-null  float64
 11  Marital_Status           11048 non-null  object 
 12  rev_per_month            11158 non-null  object 
 13

In [10]:
#Check for duplicate values
print('****Duplicates data in the train dataset without EmployeeID Column****')
print('**********************************************************************')

dups = df.loc[:, ~df.columns. isin(['EmployeeID'])].duplicated().sum()
if dups ==0:
    print('There is no duplicate values in the data.') 
else:
    print('There are {} duplicates in the data which is {}% of total data'.format(dups,round((dups/df.shape[0])*100,2)))

****Duplicates data in the train dataset without EmployeeID Column****
**********************************************************************
There is no duplicate values in the data.


In [22]:
#Check for missing value
print('****Missing Values in the Dataset****')
print('*************************************')
msv = df.isnull().sum()[df.isnull().sum()>0]
if msv.empty:
    print('There is no missing values in the data.') 
else:
    for i in range(msv.count()):
        print('{} Missing values in {} which is {}% of total data ({})'.format(msv[i],msv.index[i],round(((msv[i]/df.shape[0])*100),2),df[msv.index[i]].dtype))

****Missing Values in the Dataset****
*************************************
102 Missing values in Tenure which is 0.91% of total data (object)
112 Missing values in City_Tier which is 0.99% of total data (float64)
102 Missing values in CC_Contacted_LY which is 0.91% of total data (float64)
109 Missing values in Payment which is 0.97% of total data (object)
108 Missing values in Gender which is 0.96% of total data (object)
98 Missing values in Service_Score which is 0.87% of total data (float64)
112 Missing values in Account_user_count which is 0.99% of total data (object)
97 Missing values in account_segment which is 0.86% of total data (object)
116 Missing values in CC_Agent_Score which is 1.03% of total data (float64)
212 Missing values in Marital_Status which is 1.88% of total data (object)
102 Missing values in rev_per_month which is 0.91% of total data (object)
357 Missing values in Complain_ly which is 3.17% of total data (float64)
357 Missing values in Day_Since_CC_connect which

In [46]:
# Special charectors int the numeric columns
Vars=['Tenure','Account_user_count','rev_per_month','coupon_used_for_payment','Day_Since_CC_connect','cashback']

for i in Vars:
    li=df[i].unique()
    print(i,'\t',list(filter(lambda x: (str(x).replace('.','').isdigit()!=True and str(x)!='nan'),li)))

Tenure 	 ['#']
Account_user_count 	 ['@']
rev_per_month 	 ['+']
coupon_used_for_payment 	 ['#', '$', '*']
Day_Since_CC_connect 	 ['$']
cashback 	 ['$']


In [47]:
# Special charectors int the text columns
Vars=['account_segment']

for i in Vars:
    li=df[i].unique()
    print(i,'\t',list(filter(lambda x: (str(x).replace('.','').isdigit()!=True and str(x)!='nan'),li)))

account_segment 	 ['Super', 'Regular Plus', 'Regular', 'HNI', 'Regular +', 'Super Plus', 'Super +']
